In [1]:
import pandas as pd

In [2]:
dtypes = {
    '관리건축물대장PK': 'string',
    '그룹 종류': 'Int64',
    '대장구분코드': 'string',
    '대장구분코드명': 'string',
    '대장종류코드': 'string',
    '대장종류코드명': 'string',
    '신구대장구분코드': 'string',
    '신구대장구분코드명': 'string',
    '대지위치': 'string',
    '도로명대지위치': 'string',
    '건물명': 'string',
    '시군구코드': 'string',
    '법정동코드': 'string',
    '대지구분코드': 'string',
    '번': 'string',
    '지': 'string',
    '새주소도로코드': 'string',
    '새주소법정동코드': 'string',
    '새주소지상지하코드': 'string',
    '새주소본번': 'Int64',
    '새주소부번': 'Int64',
    '대지면적(㎡)': 'Float64',
    '건축면적(㎡)': 'Float64',
    '건폐율(%)': 'Float64',
    '연면적(㎡)': 'Float64',
    '용적률산정연면적(㎡)': 'Float64',
    '용적률(%)': 'Float64',
    '주용도코드': 'string',
    '주용도코드명': 'string',
    '기타용도': 'string',
    '세대수(세대)': 'Int64',
    '가구수(가구)': 'Int64',
    '허가일': 'string',
    '착공일': 'string',
    '사용승인일': 'string',
    '호수(호)': 'Int64',
    '에너지효율등급': 'string',
    '에너지절감율': 'Float64',
    'EPI점수': 'Float64',
    '친환경건축물등급': 'string',
    '친환경건축물인증점수': 'Float64',
    '지능형건축물등급': 'string',
    '지능형건축물인증점수': 'Float64',
    '생성일자': 'string',
    '동명칭': 'string',
    '주부속구분코드': 'string',
    '주부속구분코드명': 'string',
    '구조코드': 'string',
    '구조코드명': 'string',
    '기타구조': 'string',
    '지붕코드': 'string',
    '지붕코드명': 'string',
    '기타지붕': 'string',
    '높이(m)': 'Float64',
    '지상층수': 'Int64',
    '지하층수': 'Int64',
    '승용승강기수': 'Int64',
    '비상용승강기수': 'Int64',
    '총동연면적(㎡)': 'Float64',
    '내진설계적용여부': 'string',
    '내진능력': 'string'
}

# 파일 읽기

In [3]:
df = pd.read_csv(f'../input/4_건축물대장.csv', dtype=dtypes)
print(f'df : {len(df)}')

df : 613590


# 필터링

In [4]:
mask = df['주용도코드명'].str.contains('단독주택|공동주택', na=False)

filtered_df = df[mask]
filtered_df['필터링종류']="주거"
print(f"filtered_df : {len(filtered_df)}")

unfiltered_df = df[~mask]
print(f"unfiltered_df : {len(unfiltered_df)}")

filtered_df : 442615
unfiltered_df : 170975


/var/folders/tk/y1vzx6w13s9gmhsstd16yl_00000gn/T/ipykernel_35399/1789709695.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['필터링종류']="주거"


In [5]:
filtered_df_대지위치_set = set(filtered_df['대지위치'].dropna())
filtered_df_도로명대지위치_set = set(filtered_df['도로명대지위치'].dropna())

match_대지위치 = unfiltered_df[unfiltered_df['대지위치'].isin(filtered_df_대지위치_set)]
print(f"대지위치가 같아서 포함시킴 : {len(match_대지위치)}")
match_도로명대지위치 = unfiltered_df[unfiltered_df['도로명대지위치'].isin(filtered_df_도로명대지위치_set)]
print(f"도로명주소가 같아서 포함시킴 : {len(match_도로명대지위치)}")


extra_df = pd.concat([match_대지위치, match_도로명대지위치], ignore_index=True).drop_duplicates()
extra_df['필터링종류']="비주거(동일주소)"
print(f"주소가 같아서 포함시킴 : {len(extra_df)}")
result_df = pd.concat([filtered_df,extra_df], ignore_index=True)
print(f"최종 결과 : {len(result_df)}")

대지위치가 같아서 포함시킴 : 13361
도로명주소가 같아서 포함시킴 : 10842
주소가 같아서 포함시킴 : 15193
최종 결과 : 457808


# 디자인 추가

In [6]:
def color_필터종류(val):
    if val == "주거":
        return 'background-color: #cfe8ff'
    elif val == "비주거(동일주소)":
        return 'background-color: #ffcccc'
    else:
        return ''
    
result_styled = (
    result_df.style
    .map(color_필터종류, subset=['필터링종류'])
)

In [7]:
# extra_df.to_excel("../output/비주거(동일주소).xlsx", index=False)

In [9]:
result_styled.to_excel('../output/건축물대장_주용도필터_디자인추가.xlsx', index=False)